In [ ]:
import pandas as pd
import numpy as np
import sys
import re
import functools
import json
import random
import os
import math

import matplotlib.pyplot as plt

In [ ]:
#os.getcwd()

# Linux workstation
#data_path = '/home/db600/phd/data/'

# Laptop
data_path = 'C:\\Users\\dan\\Documents\\phd\\data\\'

os.listdir(data_path)

In [ ]:
mut_path = data_path + 'depmap\\OmicsSomaticMutations.csv'
exp_path = data_path + 'depmap\\OmicsExpressionProteinCodingGenesTPMLogp1.csv' 
conv_path = data_path + 'biomart\\ensembl_biomart_plus_fasta.csv'

In [ ]:
# Read the multi_gene_converter into a DF
conv = pd.read_csv(conv_path, header = 0, index_col = 0)
conv = conv.drop(columns='Unnamed: 0')

In [ ]:
conv.head()

In [ ]:
def load_list(path):
    with open(path) as f:
        g = json.load(f)
    return g

kinases_path = "C:\\Users\\dan\\PycharmProjects\\kinase-onc-tsg\\data\\kinases.json"
oncs_path = "C:\\Users\\dan\\PycharmProjects\\kinase-onc-tsg\\data\\oncs.json"
tsgs_path = "C:\\Users\\dan\\PycharmProjects\\kinase-onc-tsg\\data\\tsgs.json"

kinases = load_list(kinases_path)
oncs = load_list(oncs_path)
tsgs =load_list(tsgs_path)

In [ ]:
# Get list of training cell lines used in original program
# Manually saved to CSV earlier
cell_lines = pd.read_csv(data_path + 'dependant\\original_training_cell_lines.csv')
cell_lines = cell_lines.rename(columns={'cell_line': 'CCLEName'})
cell_lines

In [ ]:
# Load the DepMap model metadata
depmap_model = pd.read_csv(data_path + 'depmap\\Model.csv')

# Add depmap model ID to the list of 39 original training cell lines
cell_lines = pd.merge(cell_lines, depmap_model[['ModelID', 'CCLEName']], on='CCLEName', how='left')

# View the cell line list
cell_lines

In [ ]:
# Load the expression and mutation data
mutation_df = pd.read_csv(mut_path, low_memory=False)
expression_df = pd.read_csv(exp_path)

In [ ]:
# Preview expression data
expression_df.head()

In [ ]:
# Preview mutation data
mutation_df.head()

In [ ]:
mutation_df.columns

In [ ]:
mutation_df['TranscriptStrand'].value_counts()

In [ ]:
# Take a look at the mutation classes
mutation_df['VariantInfo'].unique()

In [ ]:
# Select all mutations for our cell lines of interest only: where 'ModelID' is in cell_lines['ModelID']
#mutation_df = mutation_df.loc[mutation_df['Tumor_Sample_Barcode'].map(lambda x: x in cell_lines)]

mutation_df = mutation_df[mutation_df['ModelID'].isin(cell_lines['ModelID'])]

mutation_df.head()

In [ ]:
# Check we've got the right number of ModelID values (should be 39)
mutation_df['ModelID'].unique()

In [ ]:
# Remove anything after "." in the 'Annotation_Transcript' column
mutation_df['Transcript'] = mutation_df['Transcript'].map(lambda x: x.split('.')[0])

mutation_df[['HugoSymbol', 'Transcript']].head()

In [ ]:
# Take a look at the mutation classes
mutation_df['VariantInfo'].unique()

In [ ]:
# First we separate out the badly pathogenic mutations - these are assumed to result in loss of function
pathogenic = ('FRAME_SHIFT_DEL', 'FRAME_SHIFT_INS', 'NONSENSE', 'NONSTOP', 'START_CODON_INS')

##### NOTE ######
# These are the variations that were considered pathogenic on the original version
# pathogenic = ['Frame_Shift_Del', 'Frame_Shift_Ins','Nonsense_Mutation','Nonstop_Mutation','Stop_Codon_Del']

# I will keep this the same for now (note no 'Stop_Codon_Del' class in the new data - presumably because it's the same a NONSTOP) but note:

# IN_FRAME_DEL - this is likely damaging but could be LOF or GOF -  select which depending on whether its onc, tsg or kinase?
# IN_FRAME_INS - as above
# START_CODON_INS - this is likely to prevent the translation of the protein, so LOF?
# START_CODON_SNP - this may prevent translation if the SNP switched the codon from methianine to another amino acid

# Filter mut_df to only include rows where the variant classification is in the pathogenic list defined above
pathogenic_mutations = mutation_df.loc[mutation_df['VariantInfo'].isin(pathogenic)]

pathogenic_mutations.head()

In [ ]:
# See if DepMap classification agrees that these are all likely to be LOF (it does)
pathogenic_mutations['LikelyGoF'].value_counts()

In [ ]:
pathogenic_mutations['LikelyLoF'].value_counts()

In [ ]:
# Group by Tumour_Sample_Barcode (cell-line name) so we have 39 rows (one for each cell line), and a column containing comma seperated list
# of all the highly pathogenic mutations in that sample
path_muts_per_sample = pathogenic_mutations.groupby('ModelID')['HugoSymbol'].apply(lambda x: ', '.join(x)).reset_index()

# Check there are no consecutive commas (denoting missing values)
#path_muts_per_sample[path_muts_per_sample['HugoSymbol'].str.contains(", , ")]
path_muts_per_sample

In [ ]:
# Write to csv
path_muts_per_sample.to_csv(data_path + '\\dependant\\pathogenic_mutations_per_sample.csv')

In [ ]:
# Select all rows of mutation DF mutation_df where VariantInfo = 'MISSENSE' and VariantType = 'SNP' 
# May also want to include START_CODON_SNP here later (not sure if SNPs in start codon will be covered by the tools that assess mutations - introns only?)
missense_snp = mutation_df[(mutation_df['VariantInfo']=='MISSENSE') & (mutation_df['VariantType']=='SNP')]

In [ ]:
missense_snp

In [ ]:
# Select all DNP and TNP missense mutations (in original version these are not assessed)
missense_dnp_tnp = mutation_df[(mutation_df['VariantInfo']=='MISSENSE') & ((mutation_df['VariantType'] == 'DNP') | (mutation_df['VariantType'] == 'TNP'))]
missense_dnp_tnp

In [ ]:
# Check what's in the chromosomes column
missense_snp['Chrom'].unique()

In [ ]:
# Create a VCF file with the SNP missense mutation data

# Copy a subset of columns from the missense_snp dataframe
missense_snp_vcf = missense_snp[['Chrom', 'Pos', 'DbsnpID',  'Ref', 'Alt']].copy()

# Rename the columns to match VCF format requirements
missense_snp_vcf.rename(columns={'Chrom' : '#CHROM' , 'Pos': 'POS', 'Ref': 'REF', 'Alt': 'ALT', 'DbsnpID' : 'ID'}, inplace=True)

# Remove 'chr' string from the chromosome column values - Fathmm-XF won't recognise this this 
missense_snp_vcf['#CHROM'] = missense_snp_vcf['#CHROM'].str.replace('chr', '')

# Write to CSV
missense_snp_vcf.to_csv(data_path + '\\dependant\\depmap_mutations_for_fathmm.vcf', sep='\t', index=False)

print(len(missense_snp_vcf))
missense_snp_vcf.head()

In [ ]:
# Get unique transcript IDs from missense SNP mutations dataframe
missense_snp_transcripts = pd.Series(missense_snp['Transcript'].unique(), name='TranscriptID')
missense_snp_transcripts

In [ ]:
# Get unique transcript IDs from biomart conv file
biomart_transcripts = pd.Series(conv['Transcript stable ID'].unique(), name='TranscriptID')
biomart_transcripts

In [ ]:
# See how many of the missense SNP transcript IDs are in the biomart conv file
# 7474 of 7523 SNP missesne transcripts are in the conv file (49 missing)
missense_snp_transcripts[missense_snp_transcripts.isin(biomart_transcripts)]

In [ ]:
# Get the subset of biomart data that corresponds to the missense SNP mutation transcript IDs, and select a subset of columns
mutation_snp_info = conv.loc[conv['Transcript stable ID'].isin(missense_snp_transcripts)]

In [ ]:
mutation_snp_info

In [ ]:
# See if there are any duplications (including only the subset of columns we are interested in)
mutation_snp_info[mutation_snp_info.duplicated(subset=['Gene stable ID', 'HGNC symbol','Transcript stable ID'], keep=False)]

In [ ]:
# Drop duplicates based on this subset of cols
mutation_snp_info = mutation_snp_info.drop_duplicates(subset=['Gene stable ID', 'HGNC symbol','Transcript stable ID'], keep='first')

In [ ]:
# See if there are still any duplicates with just two columns now (there are none)
mutation_snp_info[mutation_snp_info.duplicated(subset=['HGNC symbol','Transcript stable ID'], keep=False)]

In [ ]:
def get_info(row):
    """using the ready made conv, identify for each mutation the following
    ['Fasta',  'UniProtKB/Swiss-Prot ID',
       'Protein stable ID', 'transcript stable id', 'chromo', 'start', 'end',
       'wild', 'mutant']
    Throw away any rows which don't have this information as it will not be possible to find the missense status
       """
    # Regex pattern that is used to match protein mutations typically represented in a format like 'p.X123Y', where:
    #   'p.' indicates it is a protein mutation.
    #   The first ([A-Z]) is a capture group that matches the one-letter code of the original amino acid.
    #   (\d+) is a capture group that matches one or more digits, representing the position number of the amino acid in the protein sequence.
    #   The second ([A-Z]) is a capture group that matches the one-letter code of the new amino acid after the mutation.
    # Enables extraction of original and new amino acids and their position in the protein sequence from mutation strings.
    
    #print(len(row.index))
    
    code = re.compile('p.([A-Z])(\d+)([A-Z])')

    # Function to check if the amino acid (w) at a specific position (pos) in a sequence (x) matches what's expected.
    #   x is expected to be a string representing an amino acid sequence.
    #   w is the one-letter code of an amino acid.
    #   pos is the position of the amino acid in the sequence (1-indexed).

    def check(x,w,pos):
        """helper function used below to check that our fasta has the right amino acid at the right place"""
        if len(x)>=pos:
            return x[pos-1]==w
        else:
            return False

    # Select relevant columns and assig to 'columns'
    columns = ['Gene stable ID', 'HGNC symbol', 'Peptide','Protein stable ID', 'Transcript stable ID', 'UniProtKB/Swiss-Prot ID']

    # Assign 'Annotation_Transcript'and 'Protein_Change' values fore this row to enst and aa
    enst, aa, position, chrom = row[['Transcript','ProteinChange','Pos','Chrom']]

    # if the enst for this row is in biomart_transcript_ids (conv table)
    # set info = the row where 'Transcript stable ID' == enst, select the columns listed above, and remove dup rows based on the 'Gene stable ID', 'HGNC symbol','Transcript stable ID' cols
    if enst in biomart_transcripts.tolist():
        info = conv.loc[conv['Transcript stable ID']==enst][columns].drop_duplicates(subset = ['Gene stable ID', 'HGNC symbol','Transcript stable ID'])
        #info = conv.loc[conv['Transcript stable ID']==enst][columns].drop_duplicates(subset = ['UniProtKB/Swiss-Prot ID'])
        
        # keep count of number of matching conv rows returned
        num_rows.append(len(info))
        
        # if more than one row returned append it to multiple_rows DF for later review
        if len(info) > 1:
            
            global multiple_rows
            multiple_rows = pd.concat([multiple_rows, info])
            
    # Otherwise info = np.nan
    else:
        info = 'not in transcript IDs'
        
        global not_in_biomart
        not_in_biomart = pd.concat([not_in_biomart, pd.DataFrame([row.values], columns=not_in_biomart.columns)], axis=0, ignore_index=True)
                
    # print(info)

  # Check that 'info' object is a dataframe and not empty
    if type(info)==pd.DataFrame and info.shape[0]>0:

      # if 'ProteinChange' value (aa) matches the regex,
      # save the original amino acid, position and mutant amino acid as w, pos, m
      # otherwise save as np.nan
        match = code.match(aa)
        if match:
            w = match.group(1)
            pos = int(match.group(2))
            m = match.group(3)
        else:
            w,pos,m = np.nan,np.nan,np.nan

        # Use 'check' helper function to check that Fasta column of current row contains the wild type amino acid (w) in the correct position (pos)
        info0 = info.loc[info['Peptide'].map(lambda x:check(x,w,pos))]

        # If there is at least one row, save it as info1 (there will be one or none)
        if info0.shape[0]>0:
            info1 = info0.iloc[0]
        else:
            info1 = 'w not in pos'
            
            global w_not_in_pos
            w_not_in_pos = pd.concat([w_not_in_pos, pd.DataFrame([row.values], columns=w_not_in_pos.columns)], axis=0, ignore_index=True)

            
    else:
        info1 = 'dataframe was empty'

    # If info1 is a series (ie. none of the 'else' conditions above were fulfilled)
    # Add columns for 'transcript stable id' , 'wild', 'mutant' and 'pos'
    # Return info1
    if type(info1)==pd.Series:
        info1['transcript stable id']=enst
        info1['wild'],info1['mutant'],info1['pos'] = w,m,pos
        info1['position'], info1['chrom'] = position, chrom
    return info1

# Variables to keep count and store row content when more than one conv row is returned for a missense_mutations ENST
num_rows = []
#multiple_rows = pd.DataFrame(columns=review_cols)
multiple_rows = pd.DataFrame()

# Variables to keep count and store row content when ENST is not in biomart conv file, or the expected amino acid is not in the correct position in the Fasta/peptide column
not_in_biomart = pd.DataFrame(columns=missense_snp.columns)
w_not_in_pos = pd.DataFrame(columns=missense_snp.columns)

# Apply the 'get_info' function to rows of the missense_mutations dataframe
test_info = missense_snp.apply(get_info,axis=1)

In [ ]:
test_info

In [ ]:
# Look at the list containing the number of conv rows returned for each missense_mutations ENST
# e.g. how many returned one row, how many returned 2, etc
# Note: dropping duplicates based on my revised subset instead of UniProtKB/Swiss-Prot ID prevents multiple rows being returned
len(num_rows)
num_rows = pd.Series(num_rows, name='Counts')
num_rows.value_counts()

In [ ]:
# This contains all the conv rows that were returned as multiple rows matching a single missense_mutations ENST
# Note: dropping duplicates based on my revised subset instead of UniProtKB/Swiss-Prot ID prevents multiple rows being returned
multiple_rows

In [ ]:
# Check for the various conditions that would result from no data in a row
#test_info[test_info['Gene stable ID'] == 'not in transcript IDs']
test_info[test_info['Gene stable ID'] == 'w not in pos']
#test_info[test_info['Gene stable ID'] == 'dataframe was empty']
#test_info[test_info['Gene stable ID'].isnull()]
#test_info[test_info['pos'].isnull()]

In [ ]:
# View the ENSTs from the mutations file that were not in Biomart
# Manual cross-referencing with Ensembl shows these have transcript IDs have been deprecated (74)
# However - the genome browser at https://genome.ucsc.edu shows that the Ref amino acids at the given chromosome positions are valid
# As we are using the HGNC names and not Enseml transcript IDs this should be fine 
not_in_biomart.head(100)

not_in_biomart[['Transcript','Chrom', 'Pos', 'Ref', 'Alt', 'EntrezGeneID']]

In [ ]:
# See which rows had wrong amino acid in pos when the biomart fasta file was checked
# None of the rows had missing Fasta values so that's not the issue
# However - the genome browser at https://genome.ucsc.edu shows that the Ref amino acids at the given chromosome positions are valid
# As we are using the HGNC names and not Enseml transcript IDs these should be fine 
w_not_in_pos.head(100)

w_not_in_pos[['Transcript','Chrom', 'Pos', 'Ref', 'Alt', 'EntrezGeneID']]

# Line below shows the conv rows that match the ENSTs from w_not_in_pos
#conv[conv['Transcript stable ID'].isin(w_not_in_pos['Transcript'])][['HGNC symbol', 'Transcript stable ID', 'Peptide']]

In [ ]:
# Function to validate the presence of an expected wild-type amino acid at a specific position in a (Fasta) protein sequence.
# Returns true if wild type is in expected place
# This is really a duplication of the check already done within the get_info function

def check_wild(row):
    try:
        f,w,p =row[['Peptide','wild','pos']]
        w0 = f[(int(p)-1)]
        return w == w0
    except ValueError:
        return False

In [ ]:
# Apply the check_wild function to each row of the info dataframe
check = test_info.apply(check_wild, axis = 1) == False

In [ ]:
# There should be 231 because this is how many were not handled up by get_info function
# For example, the transcript ID ws not in the Biomart transcript IDs, or the wild type amino acid was not in the expected place in the Fasta (why?)
check.value_counts()

In [ ]:
test_info_invalid = test_info[test_info.apply(check_wild, axis = 1) == False]

In [ ]:
test_info_invalid

## Alpha Missense mutation analysis starts here

In [ ]:
# Load and preview the alpha missense data (primary assembly)
# We only need to load transcripts that are in the mutations_snp dataframe
alpha_missense = pd.read_csv(data_path + 'alphamissense\\AlphaMissense_hg38.tsv', skiprows=3, sep='\t')
alpha_missense = alpha_missense[alpha_missense['transcript_id'].isin(mutation_snp[''])]

In [ ]:
alpha_missense_len = len(alpha_missense)
print(alpha_missense_len)

alpha_missense.head(20)

In [ ]:
# Check these are all aligned to hg38
alpha_missense['genome'].value_counts()

In [ ]:
# Summarise the classes
alpha_missense['am_class'].value_counts()

In [ ]:
# Load and preview the alpha missense data (isoforms)
alpha_missense_isoforms = pd.read_csv(data_path + '\\alphamissense\\AlphaMissense_isoforms_hg38.tsv', skiprows=3, sep='\t')

In [ ]:
alpha_missense_isoforms_len = len(alpha_missense_isoforms)
print(alpha_missense_isoforms_len)

alpha_missense_isoforms.head()

In [ ]:
# Does the isoforms data file also contain any of the primary asssembly data as well?
# Note: yes - there is overlap of around half a million mutations between the two datasets
alpha_missense_overlap = alpha_missense_isoforms[alpha_missense_isoforms['transcript_id'].isin(alpha_missense['transcript_id'])]

alpha_missense_overlap_len = len(alpha_missense_overlap)
print(alpha_missense_overlap_len)

alpha_missense_overlap

In [ ]:
# Check the primary assembly data for a transcript that is in both files, so we can compare the data
# They appear to be largely the same, although am_pathogenicity scores do differ and the isoform file does not include uniprot IDs
# Alpha Missense says the primary file is more reliable, so we should use this first, then fill in any gaps with the isoform data
alpha_missense[alpha_missense['transcript_id']=='ENST00000614859.5']

In [ ]:
# remove some variables to save memory
del alpha_missense_overlap

# Save the non-overlapping records from alpha_missense_isoforms
alpha_missense_isoforms_filtered = alpha_missense_isoforms[~alpha_missense_isoforms['transcript_id'].isin(alpha_missense['transcript_id'])]

del alpha_missense_isoforms

# Combine the two datasets, selecting transcripts from the Isoforms file that are not in the primary assembly file
alpha_missense_complete = pd.concat([alpha_missense, alpha_missense_isoforms_filtered])

print(f'combined dataset size should be: {alpha_missense_len + alpha_missense_isoforms_len - alpha_missense_overlap_len}')
print(f'combined dataset actual size: {len(alpha_missense_complete)}')

In [ ]:
# Clean up
del alpha_missense
del alpha_missense_isoforms_filtered

In [ ]:
# Rename columns in alpha_missense_complete df to match those in missense_snp for the merging criteria
alpha_missense_complete.rename(columns={'#CHROM': 'Chrom', 'POS': 'Pos', 'REF': 'Ref', 'ALT': 'Alt'}, inplace=True)

# Now, merge the DataFrames
missense_snp_extended = missense_snp.merge(alpha_missense_complete[['Chrom', 'Pos', 'Ref', 'Alt', 'transcript_id', 'am_pathogenicity', 'am_class']], 
                on=['Chrom', 'Pos', 'Ref', 'Alt'], 
                how='left')

missense_snp_extended

In [ ]:
missense_mutations_isoforms = missense_mutations_extended[missense_mutations_extended['am_pathogenicity'].isnull()]

In [ ]:
missense_mutations_extended[(missense_mutations_extended['am_class'].isnull()) & (missense_mutations_extended['VariantType'] == 'SNP')]

In [ ]:
alpha_missense_isoforms = pd.read_csv(data_path + '\\alphamissense\\AlphaMissense_isoforms_hg38.tsv', skiprows=3, sep='\t')

In [ ]:
# Rename columns in alpha_missense_isoforms df to match those in missense_mutations for the merging criteria
alpha_missense_isoforms.rename(columns={'#CHROM': 'Chrom', 'POS': 'Pos', 'REF': 'Ref', 'ALT': 'Alt'}, inplace=True)

missense_mutations_isoforms = missense_mutations_isoforms.drop(columns=['am_pathogenicity', 'am_class'])

# Now, merge the DataFrames
missense_mutations_isoforms_extended = missense_mutations_isoforms.merge(alpha_missense_isoforms[['Chrom', 'Pos', 'Ref', 'Alt', 'am_pathogenicity', 'am_class']], 
                on=['Chrom', 'Pos', 'Ref', 'Alt'], 
                how='left')

missense_mutations_isoforms_extended.head()

In [ ]:
missense_complete = pd.concat([missense_mutations_isoforms_extended, missense_mutations_extended])

print(len(missense_mutations_extended))
print(len(missense_mutations_isoforms_extended))
print(len(missense_complete))

In [ ]:
missense_complete[~missense_complete['am_pathogenicity'].isnull()]

In [ ]:
# Why are there null values? 
# Note: because of the DNP and TNP mutations that cannot be assessed with alpha missense - they result in a NaN value
missense_complete[~missense_complete['am_pathogenicity'].isnull()]['am_class'].value_counts()

In [ ]:
# Manually checking some Fathmm-XF results against alpha missense (using on-screen results for Fathmm-XF)
# Note: they mostly match, but some predictions 'not available' for Fathmm-XF and some Alpha Missense results are 'ambiguous'
# Could use a combination of the two?
alpha_missense[(alpha_missense['Chrom']=='chr11') & (alpha_missense['Pos']==7313831)]

In [ ]:
pathogenic_missense = missense_complete[missense_complete['am_class'] == 'likely_pathogenic']

pathogenic_missense.columns

In [ ]:
pathogenic_missense[['ModelID','HugoSymbol']]

In [ ]:
print(len(pathogenic_missense[['ModelID','HugoSymbol']]))
print(len(pathogenic_missense[['ModelID','HugoSymbol']].drop_duplicates()))

In [ ]:
# Function assigns lof and gof label depending on whether protein is onc/tsg/kinase/other
def lof_gof(x):

    if x in tsgs:
        return 'lof'
    elif x in oncs:
        return 'gof'
    elif x in kinases:
        return 'gof'
    else:
        return 'lof'

In [ ]:
# Add lof_gof column and map to lof_gof function. Each mutation (row) in df3_sm will be labelled lof/gof
df3_sm['lof_gof'] = df3_sm['Protein stable ID'].map(lof_gof)